<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=18JwHT8YDIJhm1Of7wvk65XdVfWxcg4af' width="200" />
<figcaption></figcaption></center>
</figure>

# ***Resolución de problemas directos e inversos de ecuaciones diferenciales mediante técnicas de deep learning***

**Practicantes:** Fernando Fêtis y Diego Olguín. 

**Supervisor:** Hugo Carrillo.


In [1]:
#@title Instalación e importación de FEniCS

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()

 #   
try:
    import dolfin as dol
    from dolfin import (
        SubDomain, 
        Measure,
        plot,
        SubMesh,
        BoundaryMesh,
        MeshFunction,
        CompiledSubDomain,
        DOLFIN_EPS,
        FiniteElement,
        MixedElement,
        FunctionSpace,
        as_tensor,
        grad,
        dot,
        inner,
        dx,
        ds,
        TrialFunction,
        TestFunction,
        between,
        UnitSquareMesh,
        DirichletBC,
        Function,
        solve,
        interpolate,
        Expression
    )
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin as dol
    from dolfin import (
        SubDomain, 
        Measure,
        plot,
        SubMesh,
        BoundaryMesh,
        MeshFunction,
        CompiledSubDomain,
        DOLFIN_EPS,
        FiniteElement,
        MixedElement,
        FunctionSpace,
        as_tensor,
        grad,
        dot,
        inner,
        dx,
        ds,
        TrialFunction,
        TestFunction,
        between,
        UnitSquareMesh,
        DirichletBC,
        Function,
        solve,
        interpolate,
        Expression
    )

from IPython.display import clear_output, display
import dolfin.common.plotting as fenicsplot 

import os, sys, shutil

In [2]:
#@title Librerías

from shapely.affinity import affine_transform
from shapely.geometry import Point, Polygon
from shapely.ops import triangulate
from shapely.prepared import prep

import shapely 
import numpy as np
import random

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

from time import time

from scipy import stats

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### FEniCS

In [ ]:
def PoissonForward(V, mesh, f):  
  u = TrialFunction(V)
  v = TestFunction(V)

  a = inner(grad(u), grad(v))* dx 
  L = (f * v) * dx

  u = Function(V)
  solve(a == L, u)

  return u

In [ ]:
mesh = UnitSquareMesh(25, 25, "crossed")
V = FunctionSpace(mesh, "CG", 1)
f = interpolate(Expression("2*(x[0]+x[1]-1)", degree=1), V)
u = PoissonForward(V, mesh, f)

In [ ]:
plt.figure(figsize=(8,6))
c = plot(u)
plt.colorbar(c)
plt.title("FEniCS with no constraint")
plt.xlabel("x")
plt.ylabel("y")

plt.tight_layout()

plt.savefig("FEniCSBadPosed1.pdf")

plt.show()

In [ ]:
u0 = interpolate(Expression("pow(x[0],2)*(0.5 - x[0]/3) + pow(x[1],2)*(0.5 - x[1]/3)", degree=1), V)

plt.figure(figsize=(8,6))
c = plot(u0)
plt.colorbar(c)
plt.title("Solution with constant 0")
plt.xlabel("x")
plt.ylabel("y")

plt.tight_layout()

plt.savefig("SolConst0.pdf")

plt.show()

In [ ]:
dif = abs(u0 - u)

plt.figure(figsize=(8,6))
c = plot(dif)
plt.colorbar(c)
plt.title("Difference")
plt.xlabel("x")
plt.ylabel("y")

plt.tight_layout()

plt.savefig("FEniCSDifBadPosed1.pdf")

plt.show()

In [ ]:
# Build function space with Lagrange multiplier
P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
R = FiniteElement("Real", mesh.ufl_cell(), 0)
W = FunctionSpace(mesh, P1 * R)

# Define variational problem
(u, c) = TrialFunction(W)
(v, d) = TestFunction(W)
f = Expression("2*(x[0]+x[1]-1)", degree=2)
g = Expression("0", degree=2)
a = (inner(grad(u), grad(v)) + c*v + u*d)*dx
L = f*v*dx + g*v*ds

# Compute solution
w = Function(W)
solve(a == L, w)
(u_int0, c) = w.split()

# Plot solution
plt.figure(figsize=(8,6))
c = plot(u_int0)
plt.colorbar(c)
plt.xlabel("x")
plt.ylabel("y")
plt.title("FEniCS with null integral solution")

plt.tight_layout()

plt.savefig("FEniCSBadPosed1NullInt.pdf")

plt.show()

In [ ]:
mesh = UnitSquareMesh(25, 25, "crossed")
V = FunctionSpace(mesh, "CG", 1)
f = interpolate(Expression("exp(x[0])", degree=1), V)
u = PoissonForward(V, mesh, f)

In [ ]:
plt.figure(figsize=(8,6))
c = plot(u)
plt.colorbar(c)
plt.title("FEniCS with no constraint")
plt.xlabel("x")
plt.ylabel("y")

plt.tight_layout()

plt.savefig("FEniCSBadPosed2.pdf")

plt.show()

In [ ]:
# Build function space with Lagrange multiplier
P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
R = FiniteElement("Real", mesh.ufl_cell(), 0)
W = FunctionSpace(mesh, P1 * R)

# Define variational problem
(u, c) = TrialFunction(W)
(v, d) = TestFunction(W)
f = Expression("-cos(x[0]) - x[1]", degree=2)
g = Expression("0", degree=1)
a = (inner(grad(u), grad(v)) + c*v + u*d)*dx
L = f*v*dx + g*v*ds

# Compute solution
w = Function(W)
solve(a == L, w)
(u_int0, c) = w.split()

# Plot solution
plt.figure(figsize=(8,6))
c = plot(u_int0)
plt.colorbar(c)
plt.xlabel("x")
plt.ylabel("y")
plt.title("FEniCS with null integral solution")

plt.tight_layout()

plt.savefig("FEniCSBadPosed2NullInt.pdf")

plt.show()

### PINNs

In [ ]:
#@title Generación de mallas poligonales

class PolygonMesh:

  def __init__(self, vertices):

    P = shapely.Polygon(vertices)

    self.vertices = vertices
    self.geometry = shapely.geometry.polygon.orient(P)
    self.edges = self._get_edges()
    self.normals = {}
    self.samples = {}
    self.interior_subdomains = {}
    self.boundary_subdomains = {}

  def _get_edges(self):
    edges = []
    N_edges = len(self.geometry.boundary.coords)

    for i in range(N_edges-1):
      (x0, y0), (x1, y1) = self.geometry.boundary.coords[i], self.geometry.boundary.coords[i+1]
      edges.append([(x0, y0), (x1, y1)])
    
    return edges

  def add_interior_subdomain(self, subdomain, name=""):
    keys = self.interior_subdomains.keys()
    if name=="":
      name = "int"+str(len(keys)+1)
    elif name=="res":
      name = "int"+str(len(keys)+1)
      print("'res' is not a valid name for subdomain, instead using "+name)

    self.interior_subdomains[name] = subdomain
  

  def add_boundary_subdomain(self, subdomain, name=""):
    keys = self.boundary_subdomains.keys()
    if name=="":
      name = "int"+str(len(keys)+1)
    elif name=="res":
      name = "int"+str(len(keys)+1)
      print("'res' is not a valid name for subdomain, instead using "+name)

    self.boundary_subdomains[name] = subdomain


  def sample_interior(self, k_samples):

    if self.interior_subdomains != {}:
      k = sum(k_samples.values())
      self.samples['interior'] = {}

      i_points = {}

      for key in self.interior_subdomains.keys():
        i_points[key] = []

      i_points['res'] = []

      T = triangulate(self.geometry)
      triangs = [t for t in T if self.geometry.contains(t)]

      areas = []
      transforms = []
      for t in triangs:
          areas.append(t.area)
          (x0, y0), (x1, y1), (x2, y2), _ = t.exterior.coords
          transforms.append([x1 - x0, x2 - x0, y1 - y0, y2 - y0, x0, y0])

      for transform in random.choices(transforms, weights=areas, k=k):
          x, y = np.random.random(2)
          if x + y > 1:
            x, y = 1 - x, 1 - y 
            p = Point(x, y)
          else:
            p = Point(x, y)

          t_point = affine_transform(p, transform)
            
          xn, yn = t_point.coords.xy[0][0], t_point.coords.xy[1][0]
          bound = 'res'

          for key in self.interior_subdomains.keys():
            if self.interior_subdomains[key](xn, yn):
              if len(i_points[key]) < k_samples[key]:
                bound = key
              else:
                bound = ''

            if bound != '':
              i_points[bound].append(t_point)

      for key in self.interior_subdomains.keys():
        points = i_points[key]
        torch_points = torch.tensor([p.coords for p in points]).reshape(-1, 2).T
        self.samples['interior'][key] = torch_points

      torch_points = torch.tensor([p.coords for p in i_points['res']]).reshape(-1, 2).T
      self.samples['interior']['res'] = torch_points

    else:
      k = k_samples

      T = triangulate(self.geometry)
      triangs = [t for t in T if self.geometry.contains(t)]

      areas = []
      transforms = []
      points = []
      for t in triangs:
          areas.append(t.area)
          (x0, y0), (x1, y1), (x2, y2), _ = t.exterior.coords
          transforms.append([x1 - x0, x2 - x0, y1 - y0, y2 - y0, x0, y0])
      
      for transform in random.choices(transforms, weights=areas, k=k):
          x, y = np.random.random(2)
          if x + y > 1:
              p = Point(1 - x, 1 - y)
          else:
              p = Point(x, y)

          points.append(affine_transform(p, transform))

      torch_points = torch.tensor([p.coords for p in points]).reshape(-1, 2).T

      self.samples['interior'] = {'res': torch_points}

    return self.samples['interior']


  def sample_boundary(self, k_samples):

    if self.boundary_subdomains != {}:
      k = sum(k_samples.values())

      b_points, lengths, edges = {}, [], []
      N_edges = len(self.geometry.boundary.coords)

      for key in self.boundary_subdomains.keys():
        b_points[key] = []

      b_points['res'] = []

      for i in range(N_edges-1):
        (x0, y0), (x1, y1) = self.geometry.boundary.coords[i], self.geometry.boundary.coords[i+1]
        lengths.append(np.sqrt((x1-x0)**2 + (y1-y0)**2))
        edges.append([(x0, y0), (x1, y1)])

      for edge in random.choices(edges, weights=lengths, k=k):
        lambd = np.random.random()
        (x0, y0), (x1, y1) = edge
        xn, yn = (1-lambd)*x0 + lambd*x1, (1-lambd)*y0 + lambd*y1

        bound = 'res'

        for key in self.boundary_subdomains.keys():
          if self.boundary_subdomains[key](xn, yn):
            if len(b_points[key]) < k_samples[key]:
              bound = key
            else:
              bound = ''

        if bound != '':
          b_points[bound].append(Point(xn, yn))

      self.samples['boundary'] = {}

      for key in self.boundary_subdomains.keys():

        torch_points = torch.tensor([p.coords for p in b_points[key]]).reshape(-1, 2).T
        self.samples['boundary'][key] = torch_points
        self.normals[key] = self._normal_vector(torch_points[0], torch_points[1])

      torch_points = torch.tensor([p.coords for p in b_points['res']]).reshape(-1, 2).T
      self.samples['boundary']['res'] = torch_points
      self.normals['res'] = self._normal_vector(torch_points[0], torch_points[1])
      
    else:
      k = k_samples 

      b_points, lengths, edges = [], [], []
      N_edges = len(self.geometry.boundary.coords)

      for i in range(N_edges-1):
        (x0, y0), (x1, y1) = self.geometry.boundary.coords[i], self.geometry.boundary.coords[i+1]
        lengths.append(np.sqrt((x1-x0)**2 + (y1-y0)**2))
        edges.append([(x0, y0), (x1, y1)])

      for edge in random.choices(edges, weights=lengths, k=k):
        lambd = np.random.random()
        (x0, y0), (x1, y1) = edge
        b_points.append(Point((1-lambd)*x0 + lambd*x1, (1-lambd)*y0 + lambd*y1))

      torch_points = torch.tensor([p.coords for p in b_points]).reshape(-1, 2).T

      self.samples['boundary'] = {'res': torch_points}

      self.normals['res'] = self._normal_vector(torch_points[0], torch_points[1])

    return self.samples['boundary']


  def plot_samples(self):

    if self.samples == {}:
      print("No samples to plot. Generate samples first.")

    else:
      if 'interior' in self.samples.keys():
        for key in self.samples['interior'].keys():
          plt.plot(*self.samples['interior'][key], '.', label=key)

      if 'boundary' in self.samples.keys():
        for key in self.samples['boundary'].keys():
          plt.plot(*self.samples['boundary'][key], '.', label=key)

      plt.legend()
      plt.show()   


  def plot(self):
    x = np.array(self.geometry.exterior.coords.xy[0])
    y = np.array(self.geometry.exterior.coords.xy[1])

    plt.fill(x,y, facecolor='lightblue', edgecolor='blue')
    plt.show()


  def _normal_vector(self, x, y, tol=1e-3):
    E = self.edges
    N = len(E)

    normals = torch.zeros((len(x), 2))

    for i in range(len(x)):

      con_edge = torch.zeros(2)
      xi, yi = x[i], y[i]

      for j in range(N):
        edge = shapely.LineString(E[j])
        buf = edge.buffer(tol)
        if buf.contains(Point(xi.item(), yi.item())):
          con_edge = torch.tensor(E[j][1]) - torch.tensor(E[j][0])

      if torch.dot(con_edge, con_edge)==0:
        print("Point not in boundary or Point it's a vertex")
        n = torch.zeros(2)

      elif con_edge[0].item() == 0:
        n = torch.tensor([1., 0.])
        n *= torch.sign(n[0]*con_edge[1] - n[1]*con_edge[0])

      else:
        e1 = torch.tensor([0., 1.])
        n = e1 - torch.dot(e1, con_edge)/torch.dot(con_edge, con_edge) * con_edge 
        n *= torch.sign(n[0]*con_edge[1] - n[1]*con_edge[0])/torch.linalg.norm(n)

      normals[i] = n

    return normals.reshape(2, len(x))


  def grid(self, step):

    latmin, lonmin, latmax, lonmax = self.geometry.bounds
    prep_polygon = prep(self.geometry)

    points = []
    for lat in np.arange(latmin, latmax, step):
        for lon in np.arange(lonmin, lonmax, step):
            points.append(Point((round(lat,4), round(lon,4))))

    valid_points = []
    valid_points.extend(filter(prep_polygon.contains, points))

    grid_coords = torch.tensor([p.coords for p in valid_points]).reshape(-1, 2)

    x, y = grid_coords.T

    return x, y

In [ ]:
#@title Generación red neuronal y función de derivación

class NeuralNetwork(nn.Module):

    def __init__(self, n_layers, wide, use_dropout = False, p=0.01):
        super().__init__()
        self.inner_layers = nn.ModuleList([nn.Linear(2 if n == 0 else wide, wide)
                                           for n in range(n_layers - 1)])
        self.last_layer = nn.Linear(wide, 1)
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(p=p)

    def forward(self, x, t):
        input = torch.cat([x, t], axis=1)
        
        if self.use_dropout:
          for layer in self.inner_layers:
              input = nn.Tanh()(layer(input))
              input = self.dropout(input)

        else:
          for layer in self.inner_layers:
              input = nn.Tanh()(layer(input))

        input = self.last_layer(input)
        
        return input

def derivative(f, variable):
    return torch.autograd.grad(f.sum(), variable, create_graph=True)[0]

In [ ]:
vertices = torch.tensor([(0., 0.), (1., 0.), (1., 1.), (0., 1.)])
domain = PolygonMesh(vertices)

domain.plot()

In [ ]:
cos, pi = torch.cos, torch.pi
sin = torch.sin

def pde(x, y, net):
    u = net(x, y)
    u_x = derivative(u, x)
    u_y = derivative(u, y)
    u_xx = derivative(u_x, x)
    u_yy = derivative(u_y, y)
    
    return u_xx + u_yy - 2*(1-x-y)

def boundary_condition1(x, y, net):
    u = net(x, y)
    u_x = derivative(u, x)
    u_y = derivative(u, y)
    normal = domain.normals['res']
    n_x, n_y = normal[0].reshape(-1, 1), normal[1].reshape(-1, 1)
    u_n = u_x*n_x + u_y*n_y
    return u_n 

In [ ]:
def sol(x, y):
  return x**2*(1/2 - x/3) + y**2*(1/2 - y/3)

In [ ]:
np.random.seed(42)

net = NeuralNetwork(n_layers=2, wide=100, use_dropout=False, p=0.0)
optimizer = optim.LBFGS(net.parameters())

samples_collocation = 500
samples_boundary = 500

epochs = 30

samp_c = domain.sample_interior(samples_collocation)
samp_b = domain.sample_boundary(samples_boundary)

x_b1, y_b1 = samp_b['res'][0].unsqueeze(1).requires_grad_(), samp_b['res'][1].unsqueeze(1).requires_grad_()
x_c, y_c = samp_c['res'][0].unsqueeze(1).requires_grad_(), samp_c['res'][1].unsqueeze(1).requires_grad_()

target_boundary1 = torch.zeros_like(x_b1)
target_collocation = torch.zeros_like(x_c)

def calc_loss():
    u_boundary1 = boundary_condition1(x_b1, y_b1, net)
    loss_boundary1 = nn.MSELoss()(u_boundary1, target_boundary1)

    pde_collocation = pde(x_c, y_c, net)
    loss_collocation = nn.MSELoss()(pde_collocation, target_collocation)

    return loss_boundary1 + loss_collocation 

def closure():
    optimizer.zero_grad()
    loss = calc_loss()
    loss.backward()
    return loss

net.train()

print("Warmup")

print()

try:
  for epoch in range(1, epochs + 1):
      optimizer.step(closure)
      if epoch % 10 == 0:
          print(f'Epoch {epoch:3} - loss: {calc_loss():.8f}')

except KeyboardInterrupt:
  print()
  print("Interrupting warmup")
  pass

In [ ]:
x_plot, y_plot = domain.grid(0.02)

x1 = x_plot.reshape(-1, 1)
y1 = y_plot.reshape(-1, 1)

u_pred = net(x1, y1).reshape(-1).detach()

u_sol = sol(x_plot, y_plot)

error = torch.abs(u_sol - u_pred)

fig, ax = plt.subplots(1, 3, figsize=(21,7))
fig.tight_layout(pad=6.0)

levels = np.linspace(u_pred.min().item(), u_pred.max().item(), 30)
c = ax[0].tricontourf(x_plot, y_plot, u_pred, levels=levels)
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(c, cax=cax, orientation='vertical')

ax[0].set_xlabel("x")
ax[0].set_ylabel("y")
ax[0].set_title("PINN")

levels = np.linspace(u_sol.min().item(), u_sol.max().item(), 30)
c = ax[1].tricontourf(x_plot, y_plot, u_sol, levels=levels)
divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(c, cax=cax, orientation='vertical')

ax[1].set_xlabel("x")
ax[1].set_ylabel("y")
ax[1].set_title("Solution with constant 0")

levels = np.linspace(error.min().item(), error.max().item(), 30)
c = ax[2].tricontourf(x_plot, y_plot, error, levels=levels)
divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(c, cax=cax, orientation='vertical')

ax[2].set_xlabel("x")
ax[2].set_ylabel("y")
ax[2].set_title("Absolute error")

plt.savefig("PINNBadPosed1Seed42.pdf")

plt.show()

In [ ]:
np.random.seed(43)

net = NeuralNetwork(n_layers=2, wide=300, use_dropout=False, p=0.0)
optimizer = optim.LBFGS(net.parameters())

samples_collocation = 500
samples_boundary = 500

epochs = 30

samp_c = domain.sample_interior(samples_collocation)
samp_b = domain.sample_boundary(samples_boundary)

x_b1, y_b1 = samp_b['res'][0].unsqueeze(1).requires_grad_(), samp_b['res'][1].unsqueeze(1).requires_grad_()
x_c, y_c = samp_c['res'][0].unsqueeze(1).requires_grad_(), samp_c['res'][1].unsqueeze(1).requires_grad_()

target_boundary1 = torch.zeros_like(x_b1)
target_collocation = torch.zeros_like(x_c)

def calc_loss():
    u_boundary1 = boundary_condition1(x_b1, y_b1, net)
    loss_boundary1 = nn.MSELoss()(u_boundary1, target_boundary1)

    pde_collocation = pde(x_c, y_c, net)
    loss_collocation = nn.MSELoss()(pde_collocation, target_collocation)

    return loss_boundary1+ loss_collocation 

def closure():
    optimizer.zero_grad()
    loss = calc_loss()
    loss.backward()
    return loss

net.train()

print("Warmup")

print()

try:
  for epoch in range(1, epochs + 1):
      optimizer.step(closure)
      if epoch % 10 == 0:
          print(f'Epoch {epoch:3} - loss: {calc_loss():.8f}')

except KeyboardInterrupt:
  print()
  print("Interrupting warmup")
  pass

In [ ]:
x_plot, y_plot = domain.grid(0.02)

x1 = x_plot.reshape(-1, 1)
y1 = y_plot.reshape(-1, 1)

u_pred = net(x1, y1).reshape(-1).detach()

u_sol = sol(x_plot, y_plot)

error = torch.abs(u_sol - u_pred)

fig, ax = plt.subplots(1, 3, figsize=(21,7))
fig.tight_layout(pad=6.0)

levels = np.linspace(u_pred.min().item(), u_pred.max().item(), 30)
c = ax[0].tricontourf(x_plot, y_plot, u_pred, levels=levels)
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(c, cax=cax, orientation='vertical')

ax[0].set_xlabel("x")
ax[0].set_ylabel("y")
ax[0].set_title("PINN")

levels = np.linspace(u_sol.min().item(), u_sol.max().item(), 30)
c = ax[1].tricontourf(x_plot, y_plot, u_sol, levels=levels)
divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(c, cax=cax, orientation='vertical')

ax[1].set_xlabel("x")
ax[1].set_ylabel("y")
ax[1].set_title("Solution with constant 0")

levels = np.linspace(error.min().item(), error.max().item(), 30)
c = ax[2].tricontourf(x_plot, y_plot, error, levels=levels)
divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(c, cax=cax, orientation='vertical')

ax[2].set_xlabel("x")
ax[2].set_ylabel("y")
ax[2].set_title("Absolute error")

plt.savefig("PINNBadPosed1Seed43.pdf")

plt.show()

In [ ]:
def pde(x, y, net):
    u = net(x, y)
    u_x = derivative(u, x)
    u_y = derivative(u, y)
    u_xx = derivative(u_x, x)
    u_yy = derivative(u_y, y)
    
    return u_xx + u_yy + cos(x) + y

def boundary_condition1(x, y, net):
    u = net(x, y)
    u_x = derivative(u, x)
    u_y = derivative(u, y)
    normal = domain.normals['res']
    n_x, n_y = normal[0].reshape(-1, 1), normal[1].reshape(-1, 1)
    u_n = u_x*n_x + u_y*n_y
    return u_n 

In [ ]:
seed1, seed2 = 1, 3

In [ ]:
np.random.seed(seed1)

net1 = NeuralNetwork(n_layers=3, wide=100, use_dropout=False, p=0.0)
optimizer = optim.LBFGS(net1.parameters())

samples_collocation = 500
samples_boundary = 500

epochs = 30

samp_c = domain.sample_interior(samples_collocation)
samp_b = domain.sample_boundary(samples_boundary)

x_b1, y_b1 = samp_b['res'][0].unsqueeze(1).requires_grad_(), samp_b['res'][1].unsqueeze(1).requires_grad_()
x_c, y_c = samp_c['res'][0].unsqueeze(1).requires_grad_(), samp_c['res'][1].unsqueeze(1).requires_grad_()

target_boundary1 = torch.zeros_like(x_b1)
target_collocation = torch.zeros_like(x_c)

def calc_loss():
    u_boundary1 = boundary_condition1(x_b1, y_b1, net1)
    loss_boundary1 = nn.MSELoss()(u_boundary1, target_boundary1)

    pde_collocation = pde(x_c, y_c, net1)
    loss_collocation = nn.MSELoss()(pde_collocation, target_collocation)

    return loss_boundary1 + loss_collocation 

def closure():
    optimizer.zero_grad()
    loss = calc_loss()
    loss.backward()
    return loss

net.train()

print("Warmup")

print()

try:
  for epoch in range(1, epochs + 1):
      optimizer.step(closure)
      if epoch % 10 == 0:
          print(f'Epoch {epoch:3} - loss: {calc_loss():.8f}')

except KeyboardInterrupt:
  print()
  print("Interrupting warmup")
  pass

In [ ]:
np.random.seed(seed2)

net2 = NeuralNetwork(n_layers=3, wide=100, use_dropout=False, p=0.0)
optimizer = optim.LBFGS(net2.parameters())

samples_collocation = 500
samples_boundary = 500

epochs = 30

samp_c = domain.sample_interior(samples_collocation)
samp_b = domain.sample_boundary(samples_boundary)

x_b1, y_b1 = samp_b['res'][0].unsqueeze(1).requires_grad_(), samp_b['res'][1].unsqueeze(1).requires_grad_()
x_c, y_c = samp_c['res'][0].unsqueeze(1).requires_grad_(), samp_c['res'][1].unsqueeze(1).requires_grad_()

target_boundary1 = torch.zeros_like(x_b1)
target_collocation = torch.zeros_like(x_c)

def calc_loss():
    u_boundary1 = boundary_condition1(x_b1, y_b1, net2)
    loss_boundary1 = nn.MSELoss()(u_boundary1, target_boundary1)

    pde_collocation = pde(x_c, y_c, net2)
    loss_collocation = nn.MSELoss()(pde_collocation, target_collocation)

    return loss_boundary1 + loss_collocation 

def closure():
    optimizer.zero_grad()
    loss = calc_loss()
    loss.backward()
    return loss

net.train()

print("Warmup")

print()

try:
  for epoch in range(1, epochs + 1):
      optimizer.step(closure)
      if epoch % 10 == 0:
          print(f'Epoch {epoch:3} - loss: {calc_loss():.8f}')

except KeyboardInterrupt:
  print()
  print("Interrupting warmup")
  pass

In [ ]:
x_plot, y_plot = domain.grid(0.02)

x1 = x_plot.reshape(-1, 1)
y1 = y_plot.reshape(-1, 1)

u_pred1 = net1(x1, y1).reshape(-1).detach()
u_pred2 = net2(x1, y1).reshape(-1).detach()
dif = abs(u_pred1 - u_pred2)

fig, ax = plt.subplots(1, 3, figsize=(21,7))
fig.tight_layout(pad=6.0)

levels = np.linspace(u_pred1.min().item(), u_pred1.max().item(), 30)
c = ax[0].tricontourf(x_plot, y_plot, u_pred1, levels=levels)
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(c, cax=cax, orientation='vertical')

ax[0].set_xlabel("x")
ax[0].set_ylabel("y")
ax[0].set_title(f"PINN with seed {seed1}")

levels = np.linspace(u_pred2.min().item(), u_pred2.max().item(), 30)
c = ax[1].tricontourf(x_plot, y_plot, u_pred2, levels=levels)
divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(c, cax=cax, orientation='vertical')

ax[1].set_xlabel("x")
ax[1].set_ylabel("y")
ax[1].set_title(f"PINN with seed {seed2}")

levels = np.linspace(dif.min().item(), dif.max().item(), 30)
c = ax[2].tricontourf(x_plot, y_plot, dif, levels=levels)
divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(c, cax=cax, orientation='vertical')

ax[2].set_xlabel("x")
ax[2].set_ylabel("y")
ax[2].set_title(f"Difference")

plt.savefig("PINNBadPosed2.pdf")

plt.show()